In [1]:
''' Script to parse data from the excel spreadsheet into pickle objects
'''

import seaborn as sns
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import csv
import pandas as pd
from pandasql import sqldf
import pickle

pysqldf = lambda q: sqldf(q, globals())

#load completion data
comp_data = pd.read_csv("DATA-RAW/StimDump_6-18-2020.csv")
num_wells = len(comp_data['FileNumber'].unique())
print("Number of unique wells with completion data: ", num_wells)

#load well testing data
wtest_data = pd.read_csv("DATA-RAW/StimDump_6-18-2020_Prod_Test.csv")
num_wells = len(wtest_data['FileNo'].unique())
print("Number of unique wells with well testing data: ", num_wells)

#load production data
prod_data = pd.read_csv("DATA-RAW/MultiFieldsData-3.csv")
num_wells = len(prod_data['File_No'].unique())
print("Number of unique wells with production data: ", num_wells)

#get oil, water and gas production per day
prod_data['BBLS_Oil'] = prod_data['BBLS_Oil']/prod_data['Days']
prod_data['BBLS_Water'] = prod_data['BBLS_Water']/prod_data['Days']
prod_data['MCF_Prod'] = prod_data['MCF_Prod']/prod_data['Days']

#replace NaN & inf with zeros
prod_data = prod_data.fillna(0)
prod_data.replace(np.inf, 0, inplace=True)

Number of unique wells with completion data:  14638
Number of unique wells with well testing data:  14739
Number of unique wells with production data:  988


In [2]:
#placeholder matrix 
max_timesteps = 130

#keys for each well:
#'Prod' = 'BBLS_Oil'+'BBLS_Water'+'MCF_Prod'
#'Comp' = 'Vol', 'Lbs_Prop', 'MTPress', 'MTRate_N', 'Stages'
#'Loc' = 'Latitude', 'Longitude'
#'Cumm' = 'Cumm_Oil', 'Cumm_Water', 'Cumm_Gas',
attributes = ['FileNo', 'FieldName', 
              'Prod', 'Prod_Control',  
              'Cumm', 
              'Loc', 
              'Comp']

#unique well identifiers
FileNo = np.array(prod_data['FileNo'].unique()).astype('str')

#store all data
DATA = dict((fno, dict((attr, None) for attr in attributes)) for fno in FileNo)

#read production data, prepare data, sort by time
for idx, file in enumerate(FileNo):
    
    #get rows for this particular well only
    _ = prod_data[prod_data['FileNo']==file.astype('int32')]
    
    DATA[file]['FileNo'] = file
    DATA[file]['FieldName'] = _['FieldName'].iloc[0]
    
    coords = np.zeros([2,])
    coords[0] = _['Latitude'].iloc[0]
    coords[1] = _['Longitude'].iloc[0]
    DATA[file]['Loc'] = coords

    DATA[file]['Cumm'] = np.array((_['OilWaterGasCums'].iloc[0]).strip().split('|')).astype('int32')

    #production profiles
    prod_oil = np.zeros([max_timesteps,])
    prod_water = np.zeros([max_timesteps,])
    prod_gas = np.zeros([max_timesteps,])
    
    _oil = _['BBLS_Oil']
    _water = _['BBLS_Water']
    _gas = _['MCF_Prod']
    _row,_col = _.shape
    prod_oil[0:_row] = np.flip(_oil, axis=0)
    prod_water[0:_row] = np.flip(_water, axis=0)
    prod_gas[0:_row] = np.flip(_gas, axis=0)
    prod = np.stack((prod_oil, prod_water, prod_gas), axis=-1)
    DATA[file]['Prod'] = prod
    
    #create synthetic controls
    _ = np.sum((prod[:, 0], prod[:, 1], prod[:, 2]), axis=0)
    DATA[file]['Prod_Control'] = np.where(_==0, 0, 1)

In [3]:
#count for info, for now only accept wells with no intervention job
count_empty = 0
count_refrack = 0

#read completion data
for idx, file in enumerate(FileNo):
    
    #get rows for this particular well only
    _ = comp_data[comp_data['FileNumber']==file.astype('int32')]
    
    #if no row is returned (comp data not available), delete from the original DATA
    if _.empty:
        del DATA[file]
        count_empty = count_empty+1
        continue
        
    #if more than 1 row is returned (intervention job), delete from the original DATA
    #we need better ways to handle this later
    _row,_col = _.shape
    if _row > 1:
        del DATA[file]
        count_refrack = count_refrack+1
        continue
        
    #populate the completion parameters
    comps = np.zeros([5,])

    if _['Units'].iloc[0] == "Gal":
        #convert to Bbl
        comps[0] = _['Vol'].iloc[0]*0.0238095238
    else:
        comps[0] = _['Vol'].iloc[0]
    comps[1] = _['Lbs_Prop'].iloc[0]
    comps[2] = _['MTPress'].iloc[0]
    comps[3] = _['MTRate_N'].iloc[0]
    comps[4] = _['Stages'].iloc[0]
    DATA[file]['Comp'] = comps

#get the final keys for wells with prod data & comp data
keys = list(DATA.keys())

print("Num of wells removed, with no completion data : ", count_empty)
print("Num of wells removed, refracked : ", count_refrack)
print("Num of wells with prod data & comp data: ", len(keys))

Num of wells removed, with no completion data :  50
Num of wells removed, refracked :  52
Num of wells with prod data & comp data:  886


In [4]:
#dump/read functions
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [5]:
#save the raw data
save_obj(DATA, 'DATA-raw')